In [50]:
import matplotlib as mpl
mpl.use('TkAgg')
import sys, os
import pandas as pd
import numpy as np
import scipy.spatial as sc
import matplotlib.pyplot as plt
from matplotlib.path import Path


In [51]:
contig_data = pd.read_table('/home/bram/Documents/manual_binning/contig_statistics.txt')
contig_data['average_coverage'] = np.log10(contig_data['average_coverage'])
selection = contig_data['length'] > 500
contig_data_red = contig_data[selection]
fig, ax = plt.subplots(1, figsize=(15,15))
ax.scatter(contig_data_red['GC'], contig_data_red['average_coverage'])
fig.show()

In [52]:
hullpoints = plt.ginput(6, show_clicks=True)

In [53]:
hullpoints = [list(x) for x in hullpoints]
print(hullpoints)

[[48.90726019180086, 3.1000799914239354], [48.90726019180086, 0.5779141815996662], [78.18934115725548, 0.629386953228733], [78.29101504949665, 3.0691963284464956], [68.25919101503534, 3.065764810337891], [59.41356239005425, 3.065764810337891]]


In [54]:
hull = sc.ConvexHull(hullpoints)
for simplex in hull.simplices:
    print(simplex)
    print((hullpoints[simplex[0]][1], hullpoints[simplex[0]][0]))
    print((hullpoints[simplex[1]][1], hullpoints[simplex[1]][0]))
    ax.plot((hullpoints[simplex[0]][0], hullpoints[simplex[1]][0]), (hullpoints[simplex[0]][1], hullpoints[simplex[1]][1]), 'dk--')
    fig.show()

[3 0]
(3.0691963284464956, 78.29101504949665)
(3.1000799914239354, 48.90726019180086)
[1 0]
(0.5779141815996662, 48.90726019180086)
(3.1000799914239354, 48.90726019180086)
[2 3]
(0.629386953228733, 78.18934115725548)
(3.0691963284464956, 78.29101504949665)
[2 1]
(0.629386953228733, 78.18934115725548)
(0.5779141815996662, 48.90726019180086)


In [55]:
hull_path = Path(hullpoints)
i = 0
for x,y in zip(contig_data_red['GC'], contig_data_red['average_coverage']):
    if hull_path.contains_point((x,y)):
        i += 1
        ax.scatter(x,y, color='red')
        fig.show()
print(i)

3298


In [56]:
ID_file = open('/home/bram/Documents/manual_binning/hull_IDs.txt', 'w')
for index, row in contig_data_red.iterrows():
    if hull_path.contains_point((row['GC'], row['average_coverage'])):
        print(row['contig'], file=ID_file)
ID_file.close()
        